In [1]:
# Generated code -- http://www.redblobgames.com/grids/hexagons/
import collections
import math, random
import itertools

In [2]:
random.seed(1356) # set seed so that each play is the same

In [3]:
def hex_add(a, b):
    return Hex[a[0] + b[0], a[1] + b[1], a[2] + b[2]]

def hex_subtract(a, b):
    return Hex[a[0] - b[0], a[1] - b[1], a[2] - b[2]]

def hex_neighbor(hex, direction):
    return hex_add(hex, hex_directions[direction])

def hex_diagonal_neighbor(hex, direction):
    return hex_add(hex, hex_diagonals[direction])

def hex_length(hex):
    return (abs(hex[0]) + abs(hex[1]) + abs(hex[2])) // 2

def hex_distance(a, b):
    return hex_length(hex_subtract(a, b))

In [5]:
hex_directions = [(1, 0, -1), (1, -1, 0), (0, -1, 1), (-1, 0, 1), (-1, 1, 0), (0, 1, -1)]
hex_diagonals = [(2, -1, -1), (1, -2, 1), (-1, -1, 2), (-2, 1, 1), (-1, 2, -1), (1, 1, -2)]

In [159]:
random.seed(1356)
side = [random.getrandbits(64), random.getrandbits(64)]#[b, w, empty] # side to move ['b', 'w']#

N = 1 # Number of hexes side
p = 2 # number of players

blackstones = 136
whitestones = 136

#Game ends if number of free tiles < 4
total_nr_stones = blackstones + whitestones <4
side

[14024079292171647447, 15846705576740111]

In [160]:
def initializeBoard():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.randint(0,1)
            z = -x-y
            Hex[(x, y, z)] = ''#side[rnd]
    return Hex

board = initializeBoard()
board, len(board)

({(-1, 0, 1): '',
  (-1, 1, 0): '',
  (0, -1, 1): '',
  (0, 0, 0): '',
  (0, 1, -1): '',
  (1, -1, 0): '',
  (1, 0, -1): ''},
 7)

In [136]:
def allWhiteBoardZobrist():
    whitezobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            whitezobristboard[(x, y, z)] = rnd
    return whitezobristboard

# wzobristboard

In [135]:
def allBlackBoardZobrist():
    blackzobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            blackzobristboard[(x, y, z)] = rnd
    return blackzobristboard


# bzobristboard

In [134]:
def initializeBoardZobrist():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            Hex[(x, y, z)] = rnd
    return Hex


# zboard

In [13]:
def boardToZobrist(board, zboard, wzobristboard, bzobristboard):
    random.seed(1356)
    zobristkey = random.getrandbits(64)
    wzobrists = wzobristboard.values()
    bzobrists = bzobristboard.values()
    
    for cel in board:
        if board[cel] == 'b':
            zobristkey ^= (bzobristboard[cel] ^ zboard[cel])
        elif board[cel] == 'w':
            zobristkey ^= (wzobristboard[cel] ^ zboard[cel])
        
    return zobristkey

boardToZobrist(board, zboard, wzobristboard, bzobristboard)

14024079292171647447

In [130]:
board = initializeBoard() # board with assigned black and white pieces
zboard = initializeBoardZobrist() # initial board, without any pieces

wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

In [133]:
wzobristboard

{(-2, 0, 2): 9362942470193770445,
 (-2, 1, 1): 16273422620475297486,
 (-1, -1, 2): 13900611078549723731,
 (-1, 0, 1): 4351333299647005543,
 (-1, 1, 0): 18110085144722047360,
 (0, -2, 2): 15347246035844667575,
 (0, -1, 1): 12695862053338445958,
 (0, 0, 0): 17718183257950281057,
 (0, 1, -1): 2440782321052373051,
 (1, -2, 1): 16639836139092998833,
 (1, -1, 0): 5649954421315377117,
 (1, 0, -1): 9749259430985899888}

In [14]:
def find_scores(board, side):   
    tot = {}
    
    listOfkeys = [key for (key, value) in board.items() if value == side]
    for lok  in listOfkeys:
        tot[lok] = side
        
    return tot

In [15]:
# Form groups
# score = find_scores(board, 'w')
def findGroups(score, board):
    new_list = []
    group = {}
    group_state = {}
    group_count = 0

    for tile in score:
        counter = 0
        group[group_count] = 0
        if not tile in new_list:
            for direction in hex_directions:
                new_tile = tuple(map(sum, zip(tile, direction)))
                if new_tile in score:
                    new_list += [new_tile] # Already seen in group

                if score[tile] == board[direction]:
                    group[group_count] += counter
                    group_state.setdefault(tile, []).append(new_tile)

                counter += 1
        group_count += 1
    return group_state

In [16]:
# Check if lists can be grouped by at least 1 common member.
def Intersection(lst1, lst2): 
    return set(lst1).intersection(lst2) 

In [17]:
# Count total of each group
# group = findGroups(score, board)

def totalgroups(group):
    count_list = []

    for tile in group:
        for tile2 in group:
            if tile != tile2:
                intersect = list(Intersection(group[tile], group[tile2]))

                if len(intersect) != 0:
                    count_list += [len(list(set(group[tile] + group[tile2])))]
    return count_list


# print(totalgroups(group))

In [109]:
board = {(-2, 0, 2): 'b',
                 (-2, 1, 1): 'w',
                 (-1, -1, 2): 'b',
                 (-1, 0, 1): 'w',
                 (-1, 1, 0): 'b',
                 (0, -2, 2): 'b',
                 (0, -1, 1): 'b',
                 (0, 0, 0): 'w',
                 (0, 1, -1): 'b',
                 (1, -2, 1): 'w',
                 (1, -1, 0): 'b',
                 (1, 0, -1): 'w'}

In [131]:
score = find_scores(board, 'b') #score of player1
score2 = find_scores(board, 'w') #score of player2

group = findGroups(score, board)
group2 = findGroups(score2, board)

count_list = totalgroups(group)
count_list2 = totalgroups(group2)

print(totalgroups(group), totalgroups(group2))

[] []


In [111]:
# multiply all group counts of the same color for final game score
from functools import reduce

def multiplyall(count_list):
    if count_list != []:
        return reduce(lambda x, y: x*y, count_list)
    return 0

def result(count_list, count_list2):
    game_total = multiplyall(count_list)
    game_total2 = multiplyall(count_list2)
    return game_total, game_total2

result(count_list, count_list2)

(2401, 0)

In [20]:
# Put pieces on board
def makeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
        
    return board

In [21]:
# Put pieces on board
def unmakeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
        
    return board

In [22]:
def isEmpty(board, position):
    if position in board:
        return len(board[position]) == 0
    else:
        return False

In [102]:
def fullBoard(board):
    number_ofcells = len(board)
    number_ofstones = 0
    for cell in board:
        if board[cell] != '':
            number_ofstones += 1

    return number_ofstones >= number_ofcells

In [24]:
#Check if a move is valid
def validMove(board, x, y):
    
    #Check if there is a piece on the location
    if not isEmpty(board, (x, y, -x-y)):
        return False
    
    #Check if the piece is on the board
    if abs(x) > N or abs(y) > N:
        return False
    
    return True

In [167]:
# #Creating entire the game tree for the play
# from copy import deepcopy
# import time
# def buildGameTree(board, depth, samples):
#     depth += 1
#     gametree = {}
#     tempscore = 10
    
# #     board0 = board # board with assigned black and white pieces
#     zboard = initializeBoardZobrist() # initial board, without any pieces

#     wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
#     bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

#     zidentity0 = boardToZobrist(board, zboard, wzobristboard, bzobristboard) # current root/parent
    
#     visited = [] # Make sure you haven't done this move yet
#     for sample in range(samples):
#         board2 = deepcopy(board) # make sure that you copy the board from the parent!!

#         #Only make a move when a move is valid
#         valid = False
#         while not valid:
#             x, y = random.randint(-N,N), random.randint(-N,N)
#             if not (x, y, -x-y) in visited:
#                 valid = validMove(x, y)
#         makeMove(board2, [x, y], 'b')
#         visited += [(x, y, -x-y)]

#         #Only make a move when a move is valid 2
#         valid = False
#         while not valid:
#             x2, y2 = random.randint(-N,N), random.randint(-N,N)
#             if not (x2, y2, -x2-y2) in visited:
#                 valid = validMove(x2, y2)
#         makeMove(board2, [x2, y2], 'w')
#         visited += [(x2, y2, -x2-y2)]

# #         board2 = initializeBoard() # board with assigned black and white pieces
#         zboard = initializeBoardZobrist() # initial board, without any pieces

#         wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
#         bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

#         zidentity = boardToZobrist(board2, zboard, wzobristboard, bzobristboard)
#         gametree.setdefault(zidentity0, []).append([zidentity, board2, depth, tempscore]) # store full board + temporary score + horizon effect?
                     
#     return gametree

# start = time.time()
# g = buildGameTree(board, 0, 13)#13)
# end = time.time()
# print('Time: ', end - start)
# g

In [157]:
#Creating entire the game tree for the play
from copy import deepcopy
import time

def buildGameTree(board, depth):
    depth += 1
    gametree = {}
    tempscore = 10
    
    zboard = initializeBoardZobrist() # initial board, without any pieces
    wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
    bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces
    zidentity0 = boardToZobrist(board, zboard, wzobristboard, bzobristboard) # current root/parent

    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    
    for i in range(len(all_cells)):
        board2 = deepcopy(board) # make sure that you copy the board from the parent!!

        #Only make a move when a move is valid
        valid = False
        
        count = 0
        while not valid:
            if count < 1:
                valid = True
            cell = all_cells[count]
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]
            
            #Make two valid moves
            if validMove(board2, x, y):
                if validMove(board2, x2, y2):
                    makeMove(board2, [x, y], 'b')

                    makeMove(board2, [x2, y2], 'w')
                    valid = True
            
                    # Create a zobrist key for the new board/childnode
                    zidentity = boardToZobrist(board2, zboard, wzobristboard, bzobristboard)

                    # If the board is full, tell the score
                    if fullBoard(board2):
                        
                        score = find_scores(board2, 'b') #score of player1
                        score2 = find_scores(board2, 'w') #score of player2

                        group = findGroups(score, board2)
                        group2 = findGroups(score2, board2)

                        count_list = totalgroups(group)
                        count_list2 = totalgroups(group2)
                        
                        player_one_total, player_two_total = result(count_list, count_list2)
                        
                        # Did player 1 win, lose or draw?
                        game_state = (('l', 'w')[player_one_total < player_two_total ], 'd')[player_one_total == player_two_total]

                        #Store the final game state
                        gametree.setdefault(zidentity0, []).append([zidentity, board2, depth, tempscore, {game_state}])
                        
                    else:
                        gametree.setdefault(zidentity0, []).append([zidentity, board2, depth, tempscore, {}])#childTrees(board2, depth)]) # store full board + temporary score + horizon effect?
            
            count += 1
#             break
        all_cells = all_cells[count:]
    return gametree

start = time.time()
g = buildGameTree(board, 0)#13)
end = time.time()
print('Time: ', end - start)
g

Time:  0.06510591506958008


{14024079292171647447: [[5105244843586393516,
   {(-2, 0, 2): 'b',
    (-2, 1, 1): 'w',
    (-2, 2, 0): '',
    (-1, -1, 2): '',
    (-1, 0, 1): '',
    (-1, 1, 0): '',
    (-1, 2, -1): '',
    (0, -2, 2): '',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1): '',
    (0, 2, -2): '',
    (1, -2, 1): '',
    (1, -1, 0): '',
    (1, 0, -1): '',
    (1, 1, -2): '',
    (2, -2, 0): '',
    (2, -1, -1): '',
    (2, 0, -2): ''},
   1,
   10,
   {}],
  [5314734701223361047,
   {(-2, 0, 2): 'b',
    (-2, 1, 1): '',
    (-2, 2, 0): 'w',
    (-1, -1, 2): '',
    (-1, 0, 1): '',
    (-1, 1, 0): '',
    (-1, 2, -1): '',
    (0, -2, 2): '',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1): '',
    (0, 2, -2): '',
    (1, -2, 1): '',
    (1, -1, 0): '',
    (1, 0, -1): '',
    (1, 1, -2): '',
    (2, -2, 0): '',
    (2, -1, -1): '',
    (2, 0, -2): ''},
   1,
   10,
   {}],
  [13968044816312213851,
   {(-2, 0, 2): 'b',
    (-2, 1, 1): '',
    (-2, 2, 0): '',
    (-1, -1, 2): 'w',
    (-1, 0

In [158]:
c = 1
threshold = round(len(g)/2)
def walkaroundTree(d, c):
    for n in d:
        
        if d[n] != {} and type(d[n]) == dict:
            walkaroundTree(d[n], c+1)
        elif type(d[n]) == list and len(d[n]) != 0:
            for node in d[n]:
                if node[4] == {}:
                    node[4] = buildGameTree(node[1], c)
                    if c >= threshold :
                        return True
                    else:
                        walkaroundTree(node[4], c+1)
                elif len(node[4]) != 0:
                    print('hi4', c)
                    return True
        else:
            return True

start = time.time()
walkaroundTree(g, c)
end = time.time()
print('Time: ', end - start)

KeyboardInterrupt: 

In [156]:
g[14024079292171647447][0]

[5105244843586393516,
 {(-2, 0, 2): 'b',
  (-2, 1, 1): 'w',
  (-2, 2, 0): '',
  (-1, -1, 2): '',
  (-1, 0, 1): '',
  (-1, 1, 0): '',
  (-1, 2, -1): '',
  (0, -2, 2): '',
  (0, -1, 1): '',
  (0, 0, 0): '',
  (0, 1, -1): '',
  (0, 2, -2): '',
  (1, -2, 1): '',
  (1, -1, 0): '',
  (1, 0, -1): '',
  (1, 1, -2): '',
  (2, -2, 0): '',
  (2, -1, -1): '',
  (2, 0, -2): ''},
 1,
 10,
 {5105244843586393516: [[6831184748022861629,
    {(-2, 0, 2): 'b',
     (-2, 1, 1): 'w',
     (-2, 2, 0): 'b',
     (-1, -1, 2): 'w',
     (-1, 0, 1): '',
     (-1, 1, 0): '',
     (-1, 2, -1): '',
     (0, -2, 2): '',
     (0, -1, 1): '',
     (0, 0, 0): '',
     (0, 1, -1): '',
     (0, 2, -2): '',
     (1, -2, 1): '',
     (1, -1, 0): '',
     (1, 0, -1): '',
     (1, 1, -2): '',
     (2, -2, 0): '',
     (2, -1, -1): '',
     (2, 0, -2): ''},
    2,
    10,
    {6831184748022861629: [[5804476164541872641,
       {(-2, 0, 2): 'b',
        (-2, 1, 1): 'w',
        (-2, 2, 0): 'b',
        (-1, -1, 2): 'w',
     

In [28]:
# def buildRemainder(board):
    
#     for node in board:
#         for node2 in board[node]:
#             node2[4] = buildGameTree(node2[1], node2[2])
            
#     return board

In [29]:
# buildRemainder(g)[14024079292171647447][0]

In [297]:
# # Build the next layers

# def childTrees (board, depth):
#     totalTree = {}
#     for board_zobrist in board:
        
#         if type(board[board_zobrist]) is list:
#             for cell in board[board_zobrist]: # [identity, tree, depth, score, nextdepth]
#                 print(cell)
#                 identity =  cell[0]
#                 oldtree = cell[1]
#                 depth = cell[2]
#                 cell[4] = [buildGameTree(oldtree, depth)]# create next depth
#                 totalTree.setdefault(identity, []).append(cell)
# #                 break
        
#     return buildGameTree(board, depth)

In [372]:
count = 0

for i in range(0, 702, 2):
#     print(i)
    count += 1
    
print(count)

351


In [300]:
totalTree = childTrees (g, depth)
totalTree

[3219349059700924725, {(-2, 0, 2): 'b', (-2, 1, 1): 'w', (-1, -1, 2): '', (-1, 0, 1): '', (-1, 1, 0): '', (0, -2, 2): '', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): '', (1, -2, 1): '', (1, -1, 0): '', (1, 0, -1): ''}, 1, 10, {}]
[17726049993231416889, {(-2, 0, 2): 'b', (-2, 1, 1): '', (-1, -1, 2): 'w', (-1, 0, 1): '', (-1, 1, 0): '', (0, -2, 2): '', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): '', (1, -2, 1): '', (1, -1, 0): '', (1, 0, -1): ''}, 1, 10, {}]
[18326586463770103571, {(-2, 0, 2): 'b', (-2, 1, 1): '', (-1, -1, 2): '', (-1, 0, 1): 'w', (-1, 1, 0): '', (0, -2, 2): '', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): '', (1, -2, 1): '', (1, -1, 0): '', (1, 0, -1): ''}, 1, 10, {}]
[7438833584794820882, {(-2, 0, 2): 'b', (-2, 1, 1): '', (-1, -1, 2): '', (-1, 0, 1): '', (-1, 1, 0): 'w', (0, -2, 2): '', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): '', (1, -2, 1): '', (1, -1, 0): '', (1, 0, -1): ''}, 1, 10, {}]
[8236606048146010604, {(-2, 0, 2): 'b', (-2, 1, 1): '', (-1, -1, 2): '', (-1, 0, 1

[18277205762859827189, {(-2, 0, 2): '', (-2, 1, 1): '', (-1, -1, 2): 'w', (-1, 0, 1): '', (-1, 1, 0): 'b', (0, -2, 2): '', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): '', (1, -2, 1): '', (1, -1, 0): '', (1, 0, -1): ''}, 1, 10, {}]
[17730429887624974047, {(-2, 0, 2): '', (-2, 1, 1): '', (-1, -1, 2): '', (-1, 0, 1): 'w', (-1, 1, 0): 'b', (0, -2, 2): '', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): '', (1, -2, 1): '', (1, -1, 0): '', (1, 0, -1): ''}, 1, 10, {}]
[8796784405075423264, {(-2, 0, 2): '', (-2, 1, 1): '', (-1, -1, 2): '', (-1, 0, 1): '', (-1, 1, 0): 'b', (0, -2, 2): 'w', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): '', (1, -2, 1): '', (1, -1, 0): '', (1, 0, -1): ''}, 1, 10, {}]
[5995762243733709037, {(-2, 0, 2): '', (-2, 1, 1): '', (-1, -1, 2): '', (-1, 0, 1): '', (-1, 1, 0): 'b', (0, -2, 2): '', (0, -1, 1): 'w', (0, 0, 0): '', (0, 1, -1): '', (1, -2, 1): '', (1, -1, 0): '', (1, 0, -1): ''}, 1, 10, {}]
[2477860696142419426, {(-2, 0, 2): '', (-2, 1, 1): '', (-1, -1, 2): '', (-1, 0, 1)

[1055278633739578305, {(-2, 0, 2): '', (-2, 1, 1): '', (-1, -1, 2): '', (-1, 0, 1): '', (-1, 1, 0): '', (0, -2, 2): '', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): 'b', (1, -2, 1): '', (1, -1, 0): 'w', (1, 0, -1): ''}, 1, 10, {}]
[4689620113922292272, {(-2, 0, 2): '', (-2, 1, 1): '', (-1, -1, 2): '', (-1, 0, 1): '', (-1, 1, 0): '', (0, -2, 2): '', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): 'b', (1, -2, 1): '', (1, -1, 0): '', (1, 0, -1): 'w'}, 1, 10, {}]
[10222748410734948052, {(-2, 0, 2): 'w', (-2, 1, 1): '', (-1, -1, 2): '', (-1, 0, 1): '', (-1, 1, 0): '', (0, -2, 2): '', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): '', (1, -2, 1): 'b', (1, -1, 0): '', (1, 0, -1): ''}, 1, 10, {}]
[6526987567677591050, {(-2, 0, 2): '', (-2, 1, 1): 'w', (-1, -1, 2): '', (-1, 0, 1): '', (-1, 1, 0): '', (0, -2, 2): '', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): '', (1, -2, 1): 'b', (1, -1, 0): '', (1, 0, -1): ''}, 1, 10, {}]
[9495414270745760006, {(-2, 0, 2): '', (-2, 1, 1): '', (-1, -1, 2): 'w', (-1, 0, 1)

{}

In [145]:
import pandas as pd
pd.DataFrame(totalTree)

,2819087354705866244
0,"[{2819087354705866244: [[2819087354705866244, ..."


In [94]:
gametree['hi']['do'] = [{'z': 'l'} ,9,8]
gametree

{'hi': {'do': [{'z': 'l'}, 9, 8]}}

In [ ]:
def getComputerMove(board, computerLetter):
    # Given a board and the computer's letter, determine where to move and return that move.
    if computerLetter == 'b':
        playerLetter = 'w'
    else:
        playerLetter = 'b'

# Checking Whether the Computer Can Win in One Move

In [ ]:
# Here is the algorithm for our Tic-Tac-Toe AI:
# First, check if we can win in the next move.
for i in range(1, 10):
    boardCopy = getBoardCopy(board)
    if isSpaceFree(boardCopy, i):
        makeMove(boardCopy, computerLetter, i)
    if isWinner(boardCopy, computerLetter):
        return i

# Checking Whether the Player Can Win in One Move

In [ ]:
# Check if the player could win on their next move and block them.
for i in range(1, 10):
    boardCopy = getBoardCopy(board)
    if isSpaceFree(boardCopy, i):
        makeMove(boardCopy, playerLetter, i)
    if isWinner(boardCopy, playerLetter):
        return i

In [ ]:
#Checking the Corner, Center, and Side Spaces (in That Order)

In [ ]:
# Try to take one of the corners, if they are free.
move = chooseRandomMoveFromList(board, [1, 3, 7, 9])
if move != None:
    return move

In [ ]:
# If all the corner spaces are taken, the chooseRandomMoveFromList() function returns None, and the execution moves on to line 113:
# Try to take the center, if it is free.
if isSpaceFree(board, 5):
    return 5

In [ ]:
#If none of the corners is available, line 114 moves on the center space if it is free. If the center space isn’t free, the execution moves on to line 117:
# Move on one of the sides.
return chooseRandomMoveFromList(board, [2, 4, 6, 8])

# Checking Whether the Board Is Full

In [ ]:
def isBoardFull(board):
# Return True if every space on the board has been taken. Otherwise, return False.
for i in range(1, 10):
    if isSpaceFree(board, i):
        return False
    return True

# The Game Loop

In [ ]:
print('Welcome to Tic-Tac-Toe!')

In [ ]:
Hex

In [ ]:
# Choosing the Player’s Mark and Who Goes First
playerLetter, computerLetter = inputPlayerLetter()
turn = whoGoesFirst()
print('The ' + turn + ' will go first.')
gameIsPlaying = True

In [ ]:
while gameIsPlaying:
    if turn == 'player':
        # Player's turn
        drawBoard(theBoard)
        move = getPlayerMove(theBoard)
        makeMove(theBoard, playerLetter, move)

    if isWinner(theBoard, playerLetter):
        drawBoard(theBoard)
        print('Hooray! You have won the game!')
        gameIsPlaying = False

    else:
        if isBoardFull(theBoard):
            drawBoard(theBoard)
            print('The game is a tie!')
            break

        else:
            turn = 'computer'

    else:
        # Computer's turn
        move = getComputerMove(theBoard, computerLetter)
        makeMove(theBoard, computerLetter, move)
        if isWinner(theBoard, computerLetter):
            drawBoard(theBoard)
            print('The computer has beaten you! You lose.')
            gameIsPlaying = False
        else:
            if isBoardFull(theBoard):
                drawBoard(theBoard)
                print('The game is a tie!')
                break
            else:
                turn = 'player'

# Asking the Player to Play Again

In [ ]:
print('Do you want to play again? (yes or no)')
if not input().lower().startswith('y'):
    break

In [ ]:
# def printBoard(board):
#     line = ''
#     space = 0
#     for y in range(0, N):
#         for x in range(-N, N*2):
#             try:
#                 line += Hex[(x, y, -x-y)]
#             except:
#                 line += '_'
        
#     return line
# printBoard(Hex)